## これは何?

初心者向け講座#1の内容から学習・予測に不要な部分を排除した notebook です。メインの処理を追い駆けたい!という時にお使いください。

## Note:

**事前学習済みモデルは利用禁止です!**

### 基本の設定

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

import pandas as pd
import numpy as np
from glob import  glob

import matplotlib.pyplot as plt
import seaborn as sns

### 各種ディレクトリの定義

In [ ]:
dataset_root = '/content/drive/MyDrive/atmaCup/#11/dataset_atmaCup11'
assert dataset_root is not None

input_dir = os.path.join(dataset_root, "inputs")
photo_dir = os.path.join(input_dir, "photos")

output_dir = os.path.join(dataset_root, "output_techniques_ver1")
os.makedirs(output_dir, exist_ok=True)

train_df = pd.read_csv(os.path.join(input_dir, 'train.csv'))
test_df = pd.read_csv(os.path.join(input_dir, 'test.csv'))

material_df = pd.read_csv(os.path.join(input_dir, 'materials.csv'))
technique_df = pd.read_csv(os.path.join(input_dir, 'techniques.csv'))

In [ ]:
using_models = [os.path.join(output_dir, "0_89_1.3034967494683383.pth"), os.path.join(output_dir, "1_56_1.2170564374088664.pth"), os.path.join(output_dir, "2_52_1.3331444851810008.pth"), os.path.join(output_dir, "3_97_1.2907470048285337.pth"), os.path.join(output_dir, "4_79_1.2305836895862068.pth")]
N_TTA = 100

In [ ]:
class Config:
    N_FOLDS = 5
    N_EPOCHS = 30

### 画像データの読み込み

In [ ]:
from PIL import Image

def to_img_path(object_id):
    return os.path.join(photo_dir, f'{object_id}.jpg')

def read_image(object_id):
    return Image.open(to_img_path(object_id))

In [ ]:
!pip uninstall scikit-learn
!pip install --pre --extra-index https://pypi.anaconda.org/scipy-wheels-nightly/simple scikit-learn

Uninstalling scikit-learn-1.0.dev0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/scikit_learn-1.0.dev0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/scikit_learn.libs/libgomp-f7e03b3e.so.1.0.0
    /usr/local/lib/python3.7/dist-packages/sklearn/*
Proceed (y/n)? y
  Successfully uninstalled scikit-learn-1.0.dev0
Looking in indexes: https://pypi.org/simple, https://pypi.anaconda.org/scipy-wheels-nightly/simple
     |████████████████████████████████| 22.9MB 1.2MB/s 


In [ ]:
!pip install timm

In [ ]:
import torch
from torch import nn
from torch.optim import Adam
from torch.optim.optimizer import Optimizer
from torch.utils import data

# torchvision
from torchvision import transforms as T
# from torchvision.models import resnet34
import timm

# scikit-learn
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedGroupKFold

In [ ]:
IMG_MEAN = [0.485, 0.456, 0.406]
IMG_STD = [0.229, 0.224, 0.225]

class AtmaDataset(data.Dataset):
    """atmaCup用にデータ読み込み等を行なうデータ・セット"""
    object_path_key = "object_path"
    label_key = "target"

    @property
    def meta_keys(self):
        retval = [self.object_path_key]

        if self.is_train:
            retval += [self.label_key]

        return retval

    def __init__(self, meta_df: pd.DataFrame, is_train=True):
        """
        args:
            meta_df: 
                画像へのパスと label 情報が含まれている dataframe
                必ず object_path に画像へのパス, target に正解ラベルが入っている必要があります
            
            is_train:
                True のとき学習用のデータ拡張を適用します.
                False の時は単に size にリサイズを行います
        """

        self.is_train = is_train
        for k in self.meta_keys:
            if k not in meta_df:
                raise ValueError("meta df must have {}".format(k))

        self.meta_df = meta_df.reset_index(drop=True)
        self.index_to_data = self.meta_df.to_dict(orient="index")

        size = (256, 256)

        additional_items = (
            [T.Resize(size)]
            if not is_train
            else [
                T.RandomVerticalFlip(),
                T.RandomHorizontalFlip(),
                T.RandomResizedCrop(size),
            ]
        )

        self.transformer = T.Compose(
            [*additional_items, T.ToTensor(), T.Normalize(mean=IMG_MEAN, std=IMG_STD)]
        )

    def __getitem__(self, index):
        data = self.index_to_data[index]

        obj_path, label = data.get(self.object_path_key), data.get(self.label_key, -1)
        img = Image.open(obj_path)
        img = self.transformer(img)
        return img, label

    def __len__(self):
        return len(self.meta_df)

In [ ]:
# CUDA を使うので確認. google colab の場合 GPU accelerator をオンにしておいてください
assert torch.cuda.is_available()

DEVICE = torch.device("cuda")

## Train / Validation Phase

In [ ]:
def train(
    model: nn.Module,
    optimizer: Optimizer,
    train_loader: data.DataLoader
) -> pd.Series:

    # train にすることで model 内の学習時にのみ有効な機構が有効になります (Dropouts Layers、BatchNorm Layers...)
    model.train()
    
    criterion = nn.MSELoss()
    
    for i, (x_i, y_i) in enumerate(train_loader):
        x_i = x_i.to(DEVICE)
        y_i = y_i.to(DEVICE).reshape(-1, 1).float()

        output = model(x_i)
        loss = criterion(output, y_i)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def predict(model: nn.Module, loader: data.DataLoader) -> np.ndarray:
    # train とは逆で model 内の学習時にのみ有効な機構がオフになります (Dropouts Layers、BatchNorm Layers...)
    model.eval()
    
    predicts = []
    
    for x_i, y_i in loader:
        
        # 明示的に勾配を計算しないように指定することができます. 
        # この関数ではモデルの更新はせずに単に出力だけを使いますので勾配は不要です.
        with torch.no_grad():
            output = model(x_i.to(DEVICE))

        predicts.extend(output.data.cpu().numpy())

    pred = np.array(predicts).reshape(-1)
    return pred


def calculate_metrics(y_true, y_pred) -> dict:
    """正解ラベルと予測ラベルから指標を計算する"""    
    return {
        'rmse': mean_squared_error(y_true, y_pred) ** .5
    }


def valid(
    model: nn.Module, 
    y_valid: np.ndarray, 
    valid_loader: data.DataLoader
) -> pd.Series:
    """検証フェーズ
    与えられたモデル・データローダを使って検証フェーズを実行。スコアの dict と予測した値を返す
    """
    
    pred = predict(model, valid_loader)
    score = calculate_metrics(y_valid, pred)
    return score, pred

## Run Fold

1. train / valid の loader 作成
2. 以下を epoch 数だけ繰り返す
    1. 学習用データで学習 
    2. 検証用データで検証スコアの算出

In [ ]:
# def calc_cv(
#     model: nn.Module, 
#     valid_df: pd.DataFrame, 
#     y_valid: np.ndarray,
#     n_tta: int) -> float:
#     """
#     train / valid に分割されたデータで学習と同時に検証を行なう
#     """
    
#     #   : 検証用の方は is_train=False にしてデータ拡張オフにする
#     valid_dataset = AtmaDataset(meta_df=valid_df, is_train=False)
#     valid_loader = data.DataLoader(valid_dataset, batch_size=256, num_workers=4)
    
#     # optimizer の定義
#     optimizer = Adam(model.parameters(), lr=1e-3)

#     best_score = float('inf')
#     best_model_path = None
#     best_model = None

#     model_scores = []

#     for epoch in range(1, n_epochs + 1):
#         print(f'start {epoch}')
        
#         # 1: 学習用データで学習を実行。学習時のロスを取得
#         train(model, optimizer, train_loader)

#         # 2: 検証データでのスコアを計算
#         score_valid, y_valid_pred = valid(model=model, valid_loader=valid_loader, y_valid=y_valid)

#         model_scores.append(score_valid['rmse'])

#         model_path = os.path.join(output_dir, str(i) + '_' + str(epoch) + '_' + str(score_valid['rmse']) + '.pth')

#         if best_score > score_valid['rmse']:
#             best_score = score_valid['rmse']
#             best_model_path = model_path
#             best_model = model.state_dict()
    
#     torch.save(best_model, best_model_path)

#     torch.save(model.state_dict(), model_path)

#     fig = plt.figure()

#     plt.plot(list(range(n_epochs)), model_scores)

#     fig.savefig(os.path.join(output_dir, "scores_(" + str(i) + ")_" + str(1) + "-" + str(n_epochs) + ").png"))

#     return best_score, best_model_path

### その他

モデル作成などの関数定義

In [ ]:
def create_model():
    model = timm.create_model('efficientnet_b0', pretrained=False)
    num_ftrs = model.classifier.in_features
    model.classifier = nn.Linear(in_features=num_ftrs, out_features=1, bias=True)
    return model
    
def create_metadata(input_df):
    out_df = input_df[['object_id']].copy()
    out_df['object_path'] = input_df['object_id'].map(to_img_path)
    
    if "target" in input_df:
        out_df["target"] = (input_df['sorting_date'] - 1550) / 100

    return out_df

# def run_test_predict(model):
#     test_meta_df = create_metadata(test_df)

#     # 学習時のデータ拡張はオフにしたいので is_train=False としている
#     test_dataset = AtmaDataset(meta_df=test_meta_df, is_train=False)
#     test_loader = data.DataLoader(dataset=test_dataset, batch_size=128, drop_last=False, num_workers=4)
    
#     y_pred = predict(model, loader=test_loader)
#     return y_pred

In [ ]:
from tqdm import tqdm

def run_test_predict(model, input_df, n_tta=0):
    # n_tta > 0 の時だけデータ拡張を on にする (is_train = True)
    is_tta_mode = n_tta > 0
    test_dataset = AtmaDataset(meta_df=input_df, is_train=is_tta_mode)
    test_loader = data.DataLoader(dataset=test_dataset, batch_size=128, drop_last=False, num_workers=4)

    predictions = []
    n_times = 1 if not is_tta_mode else n_tta
    print(f"run #{n_times} times / tta={is_tta_mode}")
    for _ in tqdm(range(n_times)):
        y_pred = predict(model, loader=test_loader)
        predictions.append(y_pred)

    return np.array(predictions).mean(axis=0)

In [ ]:
train_df.columns

Index(['object_id', 'sorting_date', 'art_series_id', 'target'], dtype='object')

In [ ]:
def total_rmse(model_scores):
    total_rmse = 0
    for model_score in model_scores:
      total_rmse += model_score ** 2
    total_rmse /= 5
    total_rmse = total_rmse ** 0.5
    print('total_rmse: ' + str(total_rmse))

In [ ]:
train_pred = np.zeros(len(train_df))

In [ ]:
train_meta_df = create_metadata(train_df)

test_predictions = []

fold = StratifiedGroupKFold(n_splits=5, shuffle=False)
cv = list(fold.split(X=train_df, y=train_df['target'], groups=train_df['art_series_id']))[:Config.N_FOLDS]

model_scores = []

for i, (idx_tr, idx_valid) in enumerate(cv):
    model = create_model()
    model.to(DEVICE)
    model_path = using_models[i]
    print(model_path)
    model.load_state_dict(torch.load(model_path))

    valid_meta_df=train_meta_df.iloc[idx_valid]
    y_valid=train_meta_df['target'].values[idx_valid]
    
    y_pred_tta = run_test_predict(model, valid_meta_df, n_tta=N_TTA)
    model_score = calculate_metrics(y_valid, y_pred_tta)
    train_pred[idx_valid] = y_pred_tta

    model_scores.append(model_score['rmse'])

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/100 [00:00<?, ?it/s]

/content/drive/MyDrive/atmaCup/#11/dataset_atmaCup11/output_techniques_ver1/0_89_1.3034967494683383.pth
run #100 times / tta=True




  1%|          | 1/100 [00:03<05:10,  3.13s/it]

  2%|▏         | 2/100 [00:06<05:06,  3.13s/it]

  3%|▎         | 3/100 [00:09<05:04,  3.14s/it]

  4%|▍         | 4/100 [00:12<05:02,  3.15s/it]

  5%|▌         | 5/100 [00:15<05:04,  3.20s/it]

  6%|▌         | 6/100 [00:19<05:03,  3.23s/it]

  7%|▋         | 7/100 [00:22<05:05,  3.29s/it]

  8%|▊         | 8/100 [00:25<05:02,  3.29s/it]

  9%|▉         | 9/100 [00:29<05:01,  3.31s/it]

 10%|█         | 10/100 [00:32<05:00,  3.34s/it]

 11%|█         | 11/100 [00:36<04:59,  3.37s/it]

 12%|█▏        | 12/100 [00:39<04:54,  3.34s/it]

 13%|█▎        | 13/100 [00:42<04:46,  3.29s/it]

 14%|█▍        | 14/100 [00:45<04:38,  3.24s/it]

 15%|█▌        | 15/100 [00:48<04:34,  3.23s/it]

 16%|█▌        | 16/100 [00:52<04:31,  3.23s/it]

 17%|█▋        | 17/100 [00:55<04:28,  3.23s/it]

 18%|█▊        | 18/100 [00:58<04:23,  3.21s/it]

 19%|█▉        | 19/100 [01:01<04:20,  3.22s/it]

 20%|██        | 20/100 [01:04<04:16,  3.21s/it]

 21%|██

/content/drive/MyDrive/atmaCup/#11/dataset_atmaCup11/output_techniques_ver1/1_56_1.2170564374088664.pth


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/100 [00:00<?, ?it/s]

...



  1%|          | 1/100 [01:02<1:43:43, 62.86s/it]

  2%|▏         | 2/100 [01:05<1:13:22, 44.92s/it]

  3%|▎         | 3/100 [01:08<52:18, 32.35s/it]  

  4%|▍         | 4/100 [01:12<37:44, 23.59s/it]

  5%|▌         | 5/100 [01:15<27:37, 17.45s/it]

  6%|▌         | 6/100 [01:18<20:37, 13.16s/it]

  7%|▋         | 7/100 [01:21<15:44, 10.16s/it]

  8%|▊         | 8/100 [01:24<12:19,  8.04s/it]

  9%|▉         | 9/100 [01:27<10:02,  6.62s/it]

 10%|█         | 10/100 [01:31<08:27,  5.64s/it]

 11%|█         | 11/100 [01:34<07:18,  4.93s/it]

 12%|█▏        | 12/100 [01:37<06:30,  4.44s/it]

 13%|█▎        | 13/100 [01:41<05:58,  4.12s/it]

 14%|█▍        | 14/100 [01:44<05:33,  3.87s/it]

 15%|█▌        | 15/100 [01:47<05:15,  3.71s/it]

 16%|█▌        | 16/100 [01:51<04:58,  3.55s/it]

 17%|█▋        | 17/100 [01:54<04:43,  3.42s/it]

 18%|█▊        | 18/100 [01:57<04:31,  3.31s/it]

 19%|█▉        | 19/100 [02:00<04:27,  3.31s/it]

 20%|██        | 20/100 [02:03<04:24,  3.31s/it]

 

/content/drive/MyDrive/atmaCup/#11/dataset_atmaCup11/output_techniques_ver1/4_79_1.2305836895862068.pth


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/100 [00:00<?, ?it/s]

run #100 times / tta=True




  1%|          | 1/100 [01:08<1:53:02, 68.51s/it]

  2%|▏         | 2/100 [01:11<1:19:50, 48.88s/it]

  3%|▎         | 3/100 [01:14<56:52, 35.18s/it]  

  4%|▍         | 4/100 [01:17<40:53, 25.56s/it]

  5%|▌         | 5/100 [01:21<29:48, 18.83s/it]

  6%|▌         | 6/100 [01:24<22:06, 14.11s/it]

  7%|▋         | 7/100 [01:27<16:45, 10.81s/it]

  8%|▊         | 8/100 [01:30<13:01,  8.49s/it]

  9%|▉         | 9/100 [01:33<10:27,  6.90s/it]

 10%|█         | 10/100 [01:36<08:37,  5.75s/it]

 11%|█         | 11/100 [01:39<07:22,  4.98s/it]

 12%|█▏        | 12/100 [01:43<06:34,  4.48s/it]

 13%|█▎        | 13/100 [01:46<06:00,  4.14s/it]

 14%|█▍        | 14/100 [01:49<05:35,  3.90s/it]

 15%|█▌        | 15/100 [01:53<05:14,  3.71s/it]

 16%|█▌        | 16/100 [01:56<05:01,  3.59s/it]

 17%|█▋        | 17/100 [01:59<04:48,  3.47s/it]

 18%|█▊        | 18/100 [02:02<04:41,  3.43s/it]

 19%|█▉        | 19/100 [02:06<04:34,  3.38s/it]

 20%|██        | 20/100 [02:09<04:22,  3.28s/it]

 

In [ ]:
model_scores

[2.4860089286516978,
 2.5721784126712675,
 2.3015093855544198,
 2.565052364562814,
 2.6445266538504035]

In [ ]:
total_rmse(model_scores)

total_rmse: 2.5165970073739508


In [45]:
test_meta_df = create_metadata(test_df)
test_meta_df['target'] = np.nan

for i in range(5):
    model = create_model()
    model.to(DEVICE)
    model_path = using_models[i]
    print(model_path)
    model.load_state_dict(torch.load(model_path))
    
    # 2. モデルで予測 (本当はローカルに保存した重みを読みだすなどするほうがあとで振り返りやすいが簡易にそのまま予測する)
    y_pred_i = run_test_predict(model, test_meta_df, n_tta=N_TTA)
    test_predictions.append(y_pred_i)
    del model

/content/drive/MyDrive/atmaCup/#11/dataset_atmaCup11/output_techniques_ver1/0_89_1.3034967494683383.pth


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/100 [00:00<?, ?it/s]

run #100 times / tta=True




  1%|          | 1/100 [06:29<10:43:11, 389.81s/it]

  2%|▏         | 2/100 [06:50<7:35:39, 278.97s/it] 

  3%|▎         | 3/100 [07:09<5:25:15, 201.19s/it]

  4%|▍         | 4/100 [07:30<3:55:16, 147.04s/it]

  5%|▌         | 5/100 [07:51<2:52:43, 109.08s/it]

  6%|▌         | 6/100 [08:10<2:08:59, 82.33s/it] 

  7%|▋         | 7/100 [08:30<1:38:38, 63.64s/it]

  8%|▊         | 8/100 [08:52<1:17:58, 50.85s/it]

  9%|▉         | 9/100 [09:11<1:02:56, 41.50s/it]

 10%|█         | 10/100 [09:31<52:24, 34.94s/it] 

 11%|█         | 11/100 [09:52<45:34, 30.73s/it]

 12%|█▏        | 12/100 [10:12<40:22, 27.52s/it]

 13%|█▎        | 13/100 [10:31<36:24, 25.10s/it]

 14%|█▍        | 14/100 [10:52<33:57, 23.69s/it]

 15%|█▌        | 15/100 [11:12<32:07, 22.68s/it]

 16%|█▌        | 16/100 [11:33<30:58, 22.13s/it]

 17%|█▋        | 17/100 [11:53<29:37, 21.42s/it]

 18%|█▊        | 18/100 [12:12<28:32, 20.89s/it]

 19%|█▉        | 19/100 [12:33<28:05, 20.81s/it]

 20%|██        | 20/100 [12:53

/content/drive/MyDrive/atmaCup/#11/dataset_atmaCup11/output_techniques_ver1/1_56_1.2170564374088664.pth
run #100 times / tta=True


...



  1%|          | 1/100 [00:21<34:57, 21.19s/it]

  2%|▏         | 2/100 [00:41<34:16, 20.99s/it]

  3%|▎         | 3/100 [01:01<33:16, 20.58s/it]

  4%|▍         | 4/100 [01:21<32:47, 20.50s/it]

  5%|▌         | 5/100 [01:42<32:32, 20.55s/it]

  6%|▌         | 6/100 [02:02<31:50, 20.33s/it]

  7%|▋         | 7/100 [02:22<31:20, 20.22s/it]

  8%|▊         | 8/100 [02:42<31:14, 20.38s/it]

  9%|▉         | 9/100 [03:02<30:39, 20.21s/it]

 10%|█         | 10/100 [03:22<30:05, 20.06s/it]

 11%|█         | 11/100 [03:43<30:13, 20.37s/it]

 12%|█▏        | 12/100 [04:03<29:44, 20.27s/it]

 13%|█▎        | 13/100 [04:23<29:13, 20.16s/it]

 14%|█▍        | 14/100 [04:44<29:07, 20.32s/it]

 15%|█▌        | 15/100 [05:04<28:51, 20.37s/it]

 16%|█▌        | 16/100 [05:25<28:51, 20.61s/it]

 17%|█▋        | 17/100 [05:45<28:14, 20.42s/it]

 18%|█▊        | 18/100 [06:05<27:38, 20.22s/it]

 19%|█▉        | 19/100 [06:26<27:34, 20.43s/it]

 20%|██        | 20/100 [06:46<27:15, 20.44s/it]

 21%|██

/content/drive/MyDrive/atmaCup/#11/dataset_atmaCup11/output_techniques_ver1/4_79_1.2305836895862068.pth




  0%|          | 0/100 [00:00<?, ?it/s]

run #100 times / tta=True




  1%|          | 1/100 [00:20<33:58, 20.59s/it]

  2%|▏         | 2/100 [00:40<33:32, 20.53s/it]

  3%|▎         | 3/100 [01:00<32:48, 20.30s/it]

  4%|▍         | 4/100 [01:20<32:20, 20.21s/it]

  5%|▌         | 5/100 [01:41<32:20, 20.42s/it]

  6%|▌         | 6/100 [02:01<31:46, 20.28s/it]

  7%|▋         | 7/100 [02:22<31:30, 20.33s/it]

  8%|▊         | 8/100 [02:43<31:39, 20.64s/it]

  9%|▉         | 9/100 [03:04<31:19, 20.65s/it]

 10%|█         | 10/100 [03:24<30:56, 20.63s/it]

 11%|█         | 11/100 [03:45<30:46, 20.75s/it]

 12%|█▏        | 12/100 [04:06<30:30, 20.80s/it]

 13%|█▎        | 13/100 [04:27<30:18, 20.90s/it]

 14%|█▍        | 14/100 [04:47<29:33, 20.62s/it]

 15%|█▌        | 15/100 [05:07<29:00, 20.48s/it]

 16%|█▌        | 16/100 [05:29<28:58, 20.70s/it]

 17%|█▋        | 17/100 [05:49<28:24, 20.53s/it]

 18%|█▊        | 18/100 [06:09<27:48, 20.35s/it]

 19%|█▉        | 19/100 [06:30<27:43, 20.53s/it]

 20%|██        | 20/100 [06:50<27:09, 20.37s/it]

 21%|██

In [46]:
train_df['pred_BERT_techniques'] = train_pred

In [47]:
train_df[['object_id', 'pred_BERT_techniques']].to_csv(os.path.join(output_dir, "train_techniques.csv"), index=False)

In [48]:
# すべての予測の平均値を使う
pred_mean = np.array(test_predictions).mean(axis=0)

test_df['pred_BERT_techniques'] = pred_mean

test_df[['object_id', 'pred_BERT_techniques']].to_csv(os.path.join(output_dir, "test_techniques.csv"), index=False)